In [1]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import plotly as py
import plotly.io as pio

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

from plotly.offline import init_notebook_mode

# Show charts when running kernel
init_notebook_mode(connected=True)

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore

In [2]:
import pandas as pd

# Function to load and prepare a DataFrame from a CSV file
def load_and_prepare_df(csv_file_path, energy_type):
    df = pd.read_csv(csv_file_path)
    # Ensure the '2021' column is numeric, converting values and handling commas
    df['2021'] = pd.to_numeric(df['2021'].str.replace(',', ''), errors='coerce')
    # Rename '2021' column to reflect the energy type
    df.rename(columns={'2021': f'2021_{energy_type}'}, inplace=True)
    return df[['State', f'2021_{energy_type}']]

# Example file paths for each energy source
# Adjust these paths to your actual file locations
file_paths = {
    'petroleum': '/Users/michael/Desktop/petroleum_consumption_energy_source.csv',
    'gas': '/Users/michael/Desktop/gas_consumption_energy_source.csv',
    'nuclear': '/Users/michael/Desktop/nuclear_consumption_energy_source.csv',
    'coal': '/Users/michael/Desktop/coal_consumption_energy_source.csv',
    'renewable': '/Users/michael/Desktop/renewable_consumption_energy_source.csv'
}

# Load and prepare each DataFrame
dfs = {energy: load_and_prepare_df(path, energy) for energy, path in file_paths.items()}

# Now, iteratively merge your dataframes on 'State'
df_final = pd.DataFrame()
for df in dfs.values():
    if df_final.empty:
        df_final = df
    else:
        df_final = df_final.merge(df, on='State', how='outer')

# Calculate the total energy for each state by summing all the '2021' columns except '2021_renewable'
energy_columns = [col for col in df_final.columns if '2021' in col and 'renewable' not in col]
df_final['total_energy'] = df_final[energy_columns].sum(axis=1)

# Calculate the percentage of renewable energy
df_final['percentage_renewable'] = (df_final['2021_renewable'] / df_final['total_energy']) * 100

print(df_final[['State', 'total_energy', 'percentage_renewable']])


   State  total_energy  percentage_renewable
0     AK        659295              3.777823
1     AL       2110254             14.368555
2     AR       1047469             10.895120
3     AZ       1581720             12.462193
4     CA       5342639             24.971704
5     CO       1268161             17.037742
6     CT        776835              6.667053
7     DC         45861              7.322126
8     DE        205530              3.958546
9     FL       3834952              9.218447
10    GA       2255035             14.747798
11    HI        235676             14.895450
12    IA       1059306             56.564959
13    ID        326941             44.626400
14    IL       3757585              8.927303
15    IN       2338422              9.089164
16    KS        939019             29.468733
17    KY       1500104              6.829926
18    LA       3977893              3.641978
19    MA        906641             11.384330
20    MD        961161              7.680295
21    ME  

In [3]:
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('/Users/michael/Desktop/renewable_energy_percentage_by_state.csv')

# Convert the 'State' column to string type for consistency
df['State'] = df['State'].astype(str)

# Create the choropleth map
fig = go.Figure(data=go.Choropleth(
    locations=df['State'],  # Use the 'State' column for state codes
    z=df['percentage_renewable'].astype(float),  # Use the 'Percentage Renewable' column for the color scale
    locationmode='USA-states',  # Set the location mode to 'USA-states'
    colorscale='Blues',  # Changed the colorscale to 'Greens' for a more thematic fit
    autocolorscale=False,
    zmin=0,  # Set the minimum value of the color scale
    zmax=100,  # Set the maximum value of the color scale
    hoverinfo='none',  # This part is new
    marker_line_color='black',  # Line markers between states
    marker_line_width=0.5,  # Outline thickness
    colorbar_title="Percentage"
))

# Update the layout for the new data
fig.update_layout(
    geo=dict(
        scope='usa',  # Limit map scope to USA
        projection=go.layout.geo.Projection(type='albers usa'),  # Use Albers projection
        showlakes=True,  # Show lakes
        lakecolor='rgb(255, 255, 255)'  # Set lake color
    ),
    autosize=True,
    width=1000,
    height=500,
    font=dict(
        family="Times New Roman",
        size=18,
        color="Black"
    ),
    margin=dict(  # This part is new
        l=10,  # Left margin
        r=10,  # Right margin
        b=10,  # Bottom margin
        t=10,  # Top margin
        pad=0  # Padding
    )
)

# Show the figure
fig.show()